In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from datetime import datetime
from datetime import timedelta
from sklearn.preprocessing import StandardScaler
pd.options.mode.chained_assignment = None

# Get raw data
raw_song_data = pd.read_csv('../data/songs.csv')

# Check data types
raw_song_data.info()

In [ ]:
# Clean the data

# Remove rows and columns with less than 50% values
essential_song_data = raw_song_data.copy()

# Remove columns with less than 50% of values
remove_cols = []
for i in range(len(essential_song_data.columns)):
    if (essential_song_data[essential_song_data.columns[i]].isna().sum() >= len(essential_song_data)/2):
        remove_cols.append(essential_song_data.columns[i])
essential_song_data = essential_song_data.drop(columns=remove_cols)

# Remove rows with less than 50% of values
remove_rows = []
for i in range(len(essential_song_data)):
    if (essential_song_data.iloc[i].isna().sum() >= len(essential_song_data.columns)/2):
        remove_rows.append(i)
essential_song_data = essential_song_data.drop(index=remove_rows)

# Fix data hygiene typing issues
hygienic_song_data = essential_song_data.copy()

# Remove Title, Artist, and Index because it will not be easily passed in ML algorithm
hygienic_song_data = hygienic_song_data.drop(columns=['Title', 'Artist', 'Index'])

# Convert all month variables into ints
month_dict = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
hygienic_song_data['Month'] = hygienic_song_data['Month'].replace(month_dict)
hygienic_song_data['Month'] = hygienic_song_data['Month'].astype(int)

# Remove commas from Length Duration and cast as integer
hygienic_song_data['Length (Duration)'][hygienic_song_data['Length (Duration)'].notna()] = hygienic_song_data['Length (Duration)'][hygienic_song_data['Length (Duration)'].notna()].str.replace(',', '')
hygienic_song_data['Length (Duration)'][hygienic_song_data['Length (Duration)'].notna()] = hygienic_song_data['Length (Duration)'][hygienic_song_data['Length (Duration)'].notna()].astype(int)

# Pass all numeric data through mean imputation
imputer_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
numeric_data = hygienic_song_data.copy()
numeric_data = numeric_data.drop(columns='Top Genre')
imputer_mean.fit(numeric_data)
numeric_data[:] = imputer_mean.transform(numeric_data)

# Add age 
numeric_data['Year'][numeric_data['Year'] == 92] = 1992
numeric_data['Age'] = 0
for i in range(len(numeric_data)):
    numeric_data['Age'].iloc[i] = (datetime.now() - pd.to_datetime(f'{int(numeric_data["Year"].iloc[i])}-{int(numeric_data["Month"].iloc[i])}-1', yearfirst=True)) / timedelta(days=365)
numeric_data = numeric_data.drop(columns=['Year', 'Month'])

# Convert to standard z-scores
ss = StandardScaler()
ss.fit(numeric_data)
numeric_data[:] = ss.transform(numeric_data)

# Pass all categorical data through mode imputation
imputer_mode = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
categorical_data = pd.DataFrame(hygienic_song_data['Top Genre'].copy())
imputer_mode.fit(categorical_data)
categorical_data[:] = imputer_mode.transform(categorical_data)
categorical_data = pd.get_dummies(categorical_data, drop_first=True)

# Put data back together
hygienic_song_data = pd.concat([numeric_data, categorical_data], axis=1)